# **Catégorisez automatiquement des questions**

## partie 2/? : requête API

### <br> Un notebook de requête API, qui récupère 50 questions par requête via le wrapper StackAPI ou bien via Stack Exchange API, et qui stocke les principales caractéristiques de ces questions dans un DataFrame.

> &emsp; <br><br>


## Imports


In [56]:
import requests
import datetime
import pandas as pd


## Requete API, via Stack Exchange API


In [57]:
# "réaliser une requête de 50 questions sur une période définie, contenant le tag “python”
# et qui ont un score > 50 (votes), récupérer les données principales de la question
# (date, titre, tags, score) dans un DataFrame et les afficher"

# Authentification
# Pas nécessaire ici, nous avons droit à 10 000 requêtes(lecture)/j sans OAuth 2.0
# L' OAuth 2.0 concerne seulement les requetes en écriture


In [58]:
# RGPD
# Nous n'avons aucune raison d'extraire des données personnelles ici, donc nous eviterons non seulement
# certaines colonnes evidentes, par exemple :
# OwnerUserId, OwnerDisplayName, LastEditorUserId, LastEditorDisplayName, etc...
# mais aussi la clé primaire (l'id), qui permettrait de les retrouver
# (notre cle primaire, si nous en avons besoin, sera simplement l'index de notre dataframe)

# Nous allons donc commencer par créer un filtre.
# On peut le faire depuis https://api.stackexchange.com/docs/create-filter,
# Ou directement depuis ce notebook, en utilisant par exemple le module requests.

api_filter_url = 'https://api.stackexchange.com/2.3/filters/create'
filter_params = {
    'include': 'creation_date,title,tags,score',
    'unsafe': 'false',
}

response = requests.get(api_filter_url, params=filter_params)
data = response.json()
print(data)


{'items': [{'included_fields': ['.backoff', '.error_id', '.error_message', '.error_name', '.has_more', '.items', '.quota_max', '.quota_remaining', 'access_token.access_token', 'access_token.account_id', 'access_token.expires_on_date', 'access_token.scope', 'account_merge.merge_date', 'account_merge.new_account_id', 'account_merge.old_account_id', 'achievement.account_id', 'achievement.achievement_type', 'achievement.badge_rank', 'achievement.creation_date', 'achievement.is_unread', 'achievement.link', 'achievement.on_site', 'achievement.reputation_change', 'achievement.title', 'answer.answer_id', 'answer.collectives', 'answer.community_owned_date', 'answer.content_license', 'answer.creation_date', 'answer.is_accepted', 'answer.last_activity_date', 'answer.last_edit_date', 'answer.locked_date', 'answer.owner', 'answer.posted_by_collectives', 'answer.question_id', 'answer.recommendations', 'answer.score', 'article.article_id', 'article.article_type', 'article.creation_date', 'article.las

In [59]:
# ??

api_filter_url = 'https://api.stackexchange.com/2.3/filters/create'
filter_params = {
    'include': 'creation_date,title,tags,score',
    'unsafe': 'false',
}

response = requests.post(api_filter_url, data=filter_params)
data = response.json()
print(data)


{'items': [{'included_fields': ['.backoff', '.error_id', '.error_message', '.error_name', '.has_more', '.items', '.quota_max', '.quota_remaining', 'access_token.access_token', 'access_token.account_id', 'access_token.expires_on_date', 'access_token.scope', 'account_merge.merge_date', 'account_merge.new_account_id', 'account_merge.old_account_id', 'achievement.account_id', 'achievement.achievement_type', 'achievement.badge_rank', 'achievement.creation_date', 'achievement.is_unread', 'achievement.link', 'achievement.on_site', 'achievement.reputation_change', 'achievement.title', 'answer.answer_id', 'answer.collectives', 'answer.community_owned_date', 'answer.content_license', 'answer.creation_date', 'answer.is_accepted', 'answer.last_activity_date', 'answer.last_edit_date', 'answer.locked_date', 'answer.owner', 'answer.posted_by_collectives', 'answer.question_id', 'answer.recommendations', 'answer.score', 'article.article_id', 'article.article_type', 'article.creation_date', 'article.las

In [60]:
# Apparemment la stack exchange API inclue des champs par defaut, la doc conseille de filtrer
# manuellement la reponse


In [61]:
# Usage of /questions GET
# Gets all the questions on the site.

# interessant ?
# questions/featured     Get all questions on the site with active bounties.
# questions/unanswered   Get all questions the site considers unanswered.


In [62]:
# On peut utiliser directement l'outil en ligne, qui fournit une reponse de 100 questions max,
# format JSON compressé.
# exemple de la doc :
# /2.3/questions?fromdate=1320969600&todate=1321660800&order=desc&min=10&sort=votes&tagged=skyrim&site=gaming

# Avant de récupérer nos questions, nous pouvons d'abord nous faire une idée du nombre de questions
# qui satisfont nos critères sur une période donnée.

# Pour cela il y a plusieurs stratégies possibles,
# on peut par exemple utiiser le filtre "total" (built-in)
# Ou la méthode/endpoint "search" (plus flexible)
# Comme nous allons au final récupérer des questions, utilisons directement la méthode questions


In [63]:
# All dates in the API are in unix epoch time

def get_number_of_questions(tag, start_date, end_date, min_score=50):
    api_url = 'https://api.stackexchange.com/2.3/questions'

    params = {
        'order': 'desc',
        'sort': 'votes',
        'tagged': tag,
        'site': 'stackoverflow',
        'fromdate': int(start_date.timestamp()),
        'todate': int(end_date.timestamp()),
        'min': min_score,
        'pagesize': 100  # 30 par défaut, 100 max
    }

    response = requests.get(api_url, params=params)
    data = response.json()
    display(data)

    if 'items' in data:
        return len(data['items'])
    else:
        return 0

# Observons l'activité cette année
tag = 'python'
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 11, 27)

result = get_number_of_questions(tag, start_date, end_date)
print(f'Number of questions with the tag "{tag}" between {start_date} and {end_date} and a minimum score of 50: \n{result}')


{'items': [{'tags': ['python', 'error-handling', 'pip'],
   'owner': {'account_id': 23480036,
    'reputation': 1614,
    'user_id': 17532318,
    'user_type': 'registered',
    'profile_image': 'https://www.gravatar.com/avatar/d37672946ecec10b2814be93daa910ef?s=256&d=identicon&r=PG',
    'display_name': 'Apoliticalboy',
    'link': 'https://stackoverflow.com/users/17532318/apoliticalboy'},
   'is_answered': True,
   'view_count': 224796,
   'answer_count': 17,
   'score': 150,
   'last_activity_date': 1700813401,
   'creation_date': 1677700339,
   'last_edit_date': 1694084649,
   'question_id': 75608323,
   'content_license': 'CC BY-SA 4.0',
   'link': 'https://stackoverflow.com/questions/75608323/how-do-i-solve-error-externally-managed-environment-every-time-i-use-pip-3',
   'title': 'How do I solve &quot;error: externally-managed-environment&quot; every time I use pip 3?'},
  {'tags': ['python', 'pandas', 'dataframe', 'attributeerror'],
   'owner': {'account_id': 18651684,
    'repu

Number of questions with the tag "python" between 2023-01-01 00:00:00 and 2023-11-27 00:00:00 and a minimum score of 50: 
10


In [64]:
# Seulement 10 questions crées avec le tag python et un score > 50 (en presque un an)

# Observons l'activité l'année dernière
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

result = get_number_of_questions(tag, start_date, end_date)
print(f'Number of questions with the tag "{tag}" between {start_date} and {end_date} and a minimum score of 50: \n{result}')

# Il faut du temps pour que le score atteigne 50, mais le traffic sur stack exchange a
# peut-être aussi diminué depuis l'année dernière (arrivée de chatGPT, etc...)


{'items': [{'tags': ['python',
    'macos',
    'terminal',
    'atom-editor',
    'macos-monterey'],
   'owner': {'account_id': 24663681,
    'reputation': 3226,
    'user_id': 18559642,
    'user_type': 'registered',
    'profile_image': 'https://www.gravatar.com/avatar/13cdab4eb9d9fb5b1c7dece549f96046?s=256&d=identicon&r=PG',
    'display_name': 'Kirill Ignatyev',
    'link': 'https://stackoverflow.com/users/18559642/kirill-ignatyev'},
   'is_answered': True,
   'view_count': 370728,
   'protected_date': 1673118234,
   'accepted_answer_id': 71621142,
   'answer_count': 18,
   'score': 197,
   'last_activity_date': 1675219189,
   'creation_date': 1648058522,
   'last_edit_date': 1656277606,
   'question_id': 71591971,
   'content_license': 'CC BY-SA 4.0',
   'link': 'https://stackoverflow.com/questions/71591971/how-can-i-fix-the-zsh-command-not-found-python-error-macos-monterey-12-3',
   'title': 'How can I fix the &quot;zsh: command not found: python&quot; error? (macOS Monterey 12.

Number of questions with the tag "python" between 2022-01-01 00:00:00 and 2022-12-31 00:00:00 and a minimum score of 50: 
35


In [65]:
def get_questions(tag, start_date, end_date, min_score=50):
    api_url = 'https://api.stackexchange.com/2.3/questions'

    params = {
        'order': 'desc',
        'sort': 'votes',
        'tagged': tag,
        'site': 'stackoverflow',
        'fromdate': int(start_date.timestamp()),
        'todate': int(end_date.timestamp()),
        'min': min_score,
        'pagesize': 50  # 30 par défaut, 100 max
    }

    response = requests.get(api_url, params=params)
    data = response.json()

    return data

# Observons l'activité cette année
tag = 'python'
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2023, 11, 27)

result = get_questions(tag, start_date, end_date)
display(result)


{'items': [{'tags': ['python', 'pip', 'packaging', 'pyproject.toml'],
   'owner': {'account_id': 3400218,
    'reputation': 29218,
    'user_id': 3015186,
    'user_type': 'registered',
    'accept_rate': 50,
    'profile_image': 'https://i.stack.imgur.com/KagUg.jpg?s=256&g=1',
    'display_name': 'Niko Fohr',
    'link': 'https://stackoverflow.com/users/3015186/niko-fohr'},
   'is_answered': True,
   'view_count': 267658,
   'accepted_answer_id': 66472800,
   'answer_count': 5,
   'score': 343,
   'last_activity_date': 1677001110,
   'creation_date': 1595181079,
   'last_edit_date': 1614931841,
   'question_id': 62983756,
   'content_license': 'CC BY-SA 4.0',
   'link': 'https://stackoverflow.com/questions/62983756/what-is-pyproject-toml-file-for',
   'title': 'What is pyproject.toml file for?'},
  {'tags': ['python', 'pandas', 'xlrd', 'pcf'],
   'owner': {'account_id': 20190357,
    'reputation': 2889,
    'user_id': 14808721,
    'user_type': 'registered',
    'profile_image': 'http

In [66]:
# Plus qu'à filtrer / créer le dataframe

def extract_fields_and_create_dataframe(data):
    questions = data.get('items', [])

    # Extracting desired fields
    extracted_data = [
        {
            'creation_date': question.get('creation_date', None),
            'title': question.get('title', None),
            'tags': question.get('tags', None),
            'score': question.get('score', None),
        }
        for question in questions
    ]

    # Creating DataFrame
    df = pd.DataFrame(extracted_data)

    return df

df = extract_fields_and_create_dataframe(result)

# Displaying the DataFrame
display(df)


,creation_date,title,tags,score
0,1595181079,What is pyproject.toml file for?,"[python, pip, packaging, pyproject.toml]",343
1,1607702016,xlrd.biffh.XLRDError: Excel xlsx file; not sup...,"[python, pandas, xlrd, pcf]",287
2,1580572695,What does model.eval() do in pytorch?,"[python, machine-learning, deep-learning, pyto...",259
3,1612516308,"ValueError: numpy.ndarray size changed, may in...","[python, pandas, numpy, scikit-learn, python-3.7]",225
4,1606035195,docker.errors.DockerException: Error while fet...,"[python, linux, docker, docker-compose]",221
5,1593658606,sqlalchemy.exc.NoSuchModuleError: Can&#39;t lo...,"[python, postgresql, sqlalchemy, flask-sqlalch...",220
6,1608242137,Python was not found; run without arguments to...,"[python, python-3.x, windows-10]",203
7,1604676159,DeprecationWarning: executable_path has been d...,"[python, selenium]",198
8,1648058522,How can I fix the &quot;zsh: command not found...,"[python, macos, terminal, atom-editor, macos-m...",197
9,1587560390,ERROR: Could not build wheels for scipy which ...,"[python, scipy, pip, embedded-linux, python-wh...",196


## Préparation de la requete SQL : consultation de la doc (schema)


In [67]:
# La page d'accueil du "StackExchange Data Explorer" (l'outil d’export de données)
# présente la structure de la table, avec le type de chaque colonne, et un lien vers la doc :
# https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

# Table : posts
# "You find in Posts all non-deleted posts."

# La requete donnee en exemple est un tres bon pt de depart.
# Tags1 - question tags (PostTypeId = 1)
# AnswerCount - the number of undeleted answers (only present if PostTypeId = 1)
# FavoriteCount (nullable)

# On peut aussi y ajouter :

# extraire des questions relativement recentes : par exemple, parce que de nouveaux tags on pu apparaitre,
# qui seraient sous-estimes autrement. (c'est un domaine en ébullition !)
# CreationDate > ...



# autres pistes possibles, mais pas tres utiles je pense :

# CommentCount (nullable) : semble redondant puisqu'on a deja AnswerCount et FavoriteCount
# de + sur stack il y a generalemt bcp - de commentaires que de reponses
# ClosedDate1 (present only if the post is closed) :  pas sur que cela nous importe directement
# CommunityOwnedDate (present only if post is community wiki'd) donc pas pour les questions
# ContentLicense1 pas pertinent pour nous.





## Requete SQL


In [68]:
# SELECT TOP 50000 Title, Body, Tags, Score, ViewCount, FavoriteCount, AnswerCount
# FROM Posts
# WHERE PostTypeId = 1 AND ViewCount > 10 AND FavoriteCount > 10
# AND Score > 5 AND AnswerCount > 0 AND LEN(Tags) - LEN(REPLACE(Tags, '<','')) >= 5
# AND CreationDate >= '2020-01-01';

# est-ce qu on a vraimt besoin d'extraire les counts ?
# pourquoi pas juste Title, Body, Tags ?

# 0 row lol

# Pas besoin de GROUP BY ou d'ORDER BY ici.

# Mais pour s'amuser un peu, on peut choisir d'extraire, parmi nos questions qui remplissent nos criteres,
# les plus recentes. Notre requete devient :



